In [1]:
# !pip install python-dotenv -q
# !pip install pydantic-ai -q

In [1]:
from openai import OpenAI
import os
import random
import json
import base64
from pydantic_ai import Agent
from pydantic_ai.tools import Tool

from dependencies import Deps, Settings
from caption_agent import ImageCaptioner
from utils import (estimate_image_tokens, 
                   save_caption_output
)

from dotenv import load_dotenv
load_dotenv('.env', override=True)

True

#### Caption

In [2]:
# Get Ads Details from Context
ctx_path = r'D:\Self-Project\LLM\agent_ads\pipeline_upstream_outputs\output_with_styles_and_prompts_20250512_083938.json'

with open(ctx_path, 'r') as file:
    ctx_json = json.load(file)
    

# Get Task Type & Platform
request_details = ctx_json.get('request_details')
task_type = request_details.get('task_type')
target_platform = request_details.get('target_platform')['name']
print("="*100)
print(f"Task Type : {task_type}\n")
print(f"Target Platform : {target_platform} \n")
print("="*100)

# Get Markerting Options and Image Prompt
processing_ctx = ctx_json.get('processing_context')
marketing_options = processing_ctx.get('suggested_marketing_strategies')
image_prompts = processing_ctx.get('generated_image_prompts')

assert len(marketing_options) == len(image_prompts)
print(f"Loaded {len(image_prompts)} marketing options & image prompts.")
print(f"Marketing Options Sample : \n {marketing_options[0]} \n")
print(f"Image Prompt Sample : \n {image_prompts[0]}")
print("="*100)


Task Type : 1. Product Photography

Target Platform : Instagram Post (1:1 Square) 

Loaded 5 marketing options & image prompts.
Marketing Options Sample : 
 {'target_audience': 'Eco-conscious millennials seeking sustainable plant-based alternatives', 'target_niche': 'Plant-Based Beverages', 'target_objective': 'Increase brand awareness through engaging visual storytelling on Instagram', 'target_voice': 'Fresh, vibrant, and environmentally mindful'} 

Image Prompt Sample : 
 {'visual_concept': {'main_subject': None, 'composition_and_framing': 'The shot uses a dynamic, slightly diagonal composition to create visual momentum, placing the soy milk bottle at a 30-degree angle to suggest motion while maintaining product prominence. A cluster of macro botanicals (hydrangeas, ferns, and sprouting lentils) frames the upper left quadrant, balancing the weight of reclaimed wood textures diagonally opposite. Negative space is minimized through strategic layering of foreground foliage and backgroun

In [3]:
caption_prompt_no_context = f"""
  Given an image, create a creative, engaging, and contextually accurate Instagram caption based on the provided context and image.

  The caption is designed to intrigue viewers, spark engagement, and clearly reflect the visual content described.

  Aftter the caption, generate 5-10 relevant hashtags that are popular, engaging and directly related to the topic and visual elements, formatted neatly below the caption.
"""

In [4]:
# @title Setup User Prompt
idx = 1
image_ctx_json = image_prompts[idx]
marketing_strat = marketing_options[idx]

# Set additional prompt for each platform
target_platform = target_platform.lower()
if "instagram" in target_platform:
  additional_prompt =  """
      Prioritize brevity, visual hooks, and emotional resonance. Use short, punchy language that grabs attention quickly.
      Emojis or line breaks may be used sparingly to improve readability and expression.
      Aftter the caption, generate 5 relevant hashtags that are popular, engaging and directly related to the topic and visual elements, formatted neatly below the caption.
  """
elif "xiaohongshu" in target_platform:
  additional_prompt =  """
      Use a **first-person point of view**, as if sharing a personal experience, discovery, or recommendation.
      Provide more detailed and informative captions that feel authentic and lifestyle-oriented. Emphasize storytelling, practical value, or experience-sharing in a conversational tone.
      The caption should have a strong hook in the first line. You have to speak like a friend, not a salesperson and be honest and relatable, even with flaws or downsides.
      The caption output should be in Mandarin.
  """
elif "facebook" in target_platform:
  target_platform = 'facebook'
  additional_prompt = """
      Tailor the caption for a Facebook audience by emphasizing relatability, storytelling, and community-building.
      Begin with a strong, curiosity-piquing opening line to capture attention in the feed.
      Structure the caption with short, scannable paragraphs for readability.
      Leverage storytelling or behind-the-scenes angles when possible to deepen emotional connection.
      Subtle use of emojis and line breaks is encouraged to enhance flow without overwhelming the message.
  """

caption_prompt_with_context = f"""
  # Role and Purpose:
    You are a skilled caption-writing assistant tasked with creating compelling, platform-appropriate captions for visual content.
    You analyze both the image and strategic marketing context to produce captions that align with the visual narrative, brand tone, and engagement goals.
    Your caption should reflect the expectations and behaviors of the platform it is intended for.
    You must always ensure:
    - The caption is relevant to the image and coherent with the provided context.
    - The tone, style, and formatting match the platform conventions and target audience expectations.
    - If image details are ambiguous, make thoughtful inferences using the given context.

  # Image Context : 
    {image_ctx_json}

  # Additional information of the poster:
    - Target Audience : {marketing_strat.get('target_audience')}
    - Business Objective : {marketing_strat.get('target_objective')}
    - Content Niche/Theme : {marketing_strat.get('target_niche')}
    - Brand Voice : {marketing_strat.get('target_voice')}


  # Caption Strategy:
  - Prioritize the **Business Objective** as the foundation of the caption.
  - Use the **image context** to support and visually reinforce the marketing message — do not merely describe it.
  - Language should reflect the **Brand Voice** and speak directly to the **Target Audience's desires, challenges, or aspirations**.


  # Caption Instructions:
  1. Write a creative, engaging, and contextually relevant caption tailored for {target_platform}.
  2. Reflect the visual and strategic context accurately and creatively.
  3. Use a tone and vocabulary that aligns with the brand voice and emotional cues provided.
  4. The caption should be attention-grabbing, shareable, and foster audience interaction.
  5. Always end with a clear call to action (CTA) that encourages likes, comments, shares, or another relevant engagement.
  6. Occasionally, include a question or prompt to further invite conversation.
  7. {additional_prompt}


"""
# Not so AI , tone

print(caption_prompt_with_context)


  # Role and Purpose:
    You are a skilled caption-writing assistant tasked with creating compelling, platform-appropriate captions for visual content.
    You analyze both the image and strategic marketing context to produce captions that align with the visual narrative, brand tone, and engagement goals.
    Your caption should reflect the expectations and behaviors of the platform it is intended for.
    You must always ensure:
    - The caption is relevant to the image and coherent with the provided context.
    - The tone, style, and formatting match the platform conventions and target audience expectations.
    - If image details are ambiguous, make thoughtful inferences using the given context.

  # Image Context : 
    {'visual_concept': {'main_subject': None, 'composition_and_framing': "Central diagonal composition with the soy milk bottle tilted at 30 degrees mid-pour, capturing the liquid stream flowing from the bottle's mouth to a glass below. Sports props (matte-finish du

In [ ]:
caption_model = 'gpt-4.1-mini'
caption_agent = ImageCaptioner(model_name=caption_model)

image_path = r'D:\Self-Project\LLM\agent_ads\pipeline_donwstream_outputs\tc006\generated_image_strategy_1_20250515_143550.png'
base64_image = ImageCaptioner.encode_image(image_path)

result = caption_agent.run(
    image_base64=base64_image,
    user_prompt=caption_prompt_with_context
)

# Save output
save_caption_output(image_path, result)

Using model : gpt-4.1-mini
User Prompt: 
  # Role and Purpose:
    You are a skilled caption-writing assistant tasked with creating compelling, platform-appropriate captions for visual content.
    You analyze both the image and strategic marketing context to produce captions that align with the visual narrative, brand tone, and engagement goals.
    Your caption should reflect the expectations and behaviors of the platform it is intended for.
    You must always ensure:
    - The caption is relevant to the image and coherent with the provided context.
    - The tone, style, and formatting match the platform conventions and target audience expectations.
    - If image details are ambiguous, make thoughtful inferences using the given context.

  # Image Context : 
    {'visual_concept': {'main_subject': "A whimsical animated family of four (parents and two children) sitting at a cozy pastel-colored brunch table filled with oversized, cartoon-style menu items like a giant stack of pancak

In [ ]:
# image_tokens = estimate_image_tokens(image_path=image_path, detail='high')
# image_cost = image_tokens * 2.5 / 1000000
# print(f"Image input tokens : {image_tokens}")
# print(f"Image input cost : ${image_cost}")
# print(f"Image input cost : RM{image_cost*4.4}")

#### Test OpenAI WebSearch

In [ ]:
client = OpenAI()

# Price : gpt-4.1-mini : I 0.4/M O 1.6/M
# Price : o4-mini  : I 1.1/M O 4.4/Ms
# Price : gpt-4o-mini : I 0.15/M O 0.6/M

# search_context_size : determine how much context being retrieve to form an output
# for gpt-4.1-mini / 4o-mini
# low : 25/1k calls
# medium: 27.5/1k
# high : 30/k

# Search input template
company_name = "eesoy"
product_type = "soy milk"

search_template = f"""
  {company_name} company overview, marketing strategy, brand positioning, and notable advertising campaigns.
  If details about this company are limited, find similar brands in the {product_type} category and summarize their marketing strategy, target audience, and brand positioning instead.
"""

response = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "low",
    }],
    input=search_template
)

print(f"Search Usage = Inputs : {response.usage.input_tokens} tokens, Outputs : {response.usage.output_tokens} tokens \n")
print(f"Reasoning Effort : {response.reasoning.effort} \n")
print(f"Output : {response.output_text}")

Search Usage = Inputs : 350 tokens, Outputs : 1079 tokens 

Reasoning Effort : None 

Output : I couldn't find specific information about a company named "Eesoy." However, I can provide insights into the marketing strategies, brand positioning, and notable advertising campaigns of leading brands in the soy milk category.

**Silk Soy Milk**

- **Marketing Strategy**: Silk employs a comprehensive marketing approach that includes digital marketing, conventional advertising, and influencer partnerships. They utilize social media platforms to engage directly with consumers, answer queries, and gather feedback. Additionally, Silk integrates traditional marketing channels such as TV, magazine ads, and outdoor placements to reach a broader audience. ([case48.com](https://www.case48.com/case/Silk-Soy-Milk-A-Marketing-Strategy-11839?utm_source=openai))

- **Brand Positioning**: Silk positions itself as a health-conscious alternative to dairy milk, emphasizing the nutritional benefits of soy. The

## Sample


### *** Input : ***
peter thomas roth company overview, marketing strategy, brand positioning, and notable advertising campaigns

### *** Output : ***

Peter Thomas Roth is a skincare brand founded by its namesake, Peter Thomas Roth, a Hungarian-American entrepreneur. Established in 1993, the company is renowned for its innovative products that blend traditional European skincare techniques with modern technology.

**Company Overview:**

Peter Thomas Roth offers a comprehensive range of skincare products, including cleansers, masks, serums, moisturizers, and treatments. The brand emphasizes the use of potent ingredients such as botanicals, minerals, and vitamins to address various skin concerns. Their formulations are often inspired by European spa treatments, aiming to provide professional-level results at home.

**Marketing Strategy:**

The brand's marketing strategy focuses on educating consumers about the efficacy of its products through detailed ingredient transparency and highlighting the science behind each formulation. They leverage digital platforms, including social media and their official website, to engage with a global audience, offering skincare advice, tutorials, and customer testimonials. Collaborations with beauty influencers and professionals further enhance their reach and credibility.

**Brand Positioning:**

Peter Thomas Roth positions itself as a luxury skincare brand that delivers high-performance products inspired by European spa traditions. The brand appeals to consumers seeking effective, science-backed skincare solutions that combine the best of both worlds: traditional remedies and modern innovation.

**Notable Advertising Campaigns:**

While specific details about individual advertising campaigns are limited, Peter Thomas Roth's marketing efforts often include:

- **Influencer Partnerships:** Collaborating with beauty influencers to showcase product effectiveness and reach a broader audience.

- **Educational Content:** Providing in-depth information about product ingredients and benefits through various media channels.

- **Seasonal Promotions:** Offering limited-time deals and exclusive sets to attract both new and existing customers.

These strategies aim to build brand awareness, foster customer loyalty, and drive sales growth.

In [ ]:
# visual_context = {
#     "main_subject": "Bottle of handcrafted soy milk",
#     "composition_and_framing": "Square frame with the bottle of handcrafted soy milk placed on the right third of the image. The camera sits at a slight 10\u00b0 top-down angle to reveal the bottle\u2019s label and cap. Negative space on the left third for overlay text. Soybeans and a small ceramic bowl occupy the lower foreground, leading the eye toward the bottle.",
#     "background_environment": "A clean, bright minimalist kitchen scene: white marble countertop with subtle grey veins (#FFFFFF to #AEB0B2 gradient backdrop). In the softly blurred background, out-of-focus soy leaves and pale pastel green accents hint at plant origins without cluttering.",
#     "foreground_elements": "A scattering of fresh, golden soybeans across the countertop and a shallow off-white ceramic bowl spilling beans. A single green soy leaf rests gently beside the bowl to reinforce the natural theme.",
#     "lighting_and_mood": "High-key natural daylight from the top left, diffused through a white reflector for soft shadows. Gentle highlights on the bottle\u2019s condensation droplets create an appetizing, fresh feel. Overall mood is crisp, inviting, and wellness-oriented.",
#     "color_palette": "#FFFFFF base for background, #AEB0B2 soft grey gradient, #F06683 vibrant pink accents, #231F20 deep charcoal for text, plus natural greens from soy leaves and warm golden tones from soybeans.",
#     "visual_style": "Magazine-quality photorealism with a Scandinavian minimalist aesthetic. Shallow depth of field blurs background details. Composition uses negative space for text. Slight cinematic color grading to boost pink accent highlights and keep whites clean.",
#     "promotional_text_visuals": "Headline: \u201cPURE PLANT POWER\u201d in uppercase bold sans-serif (weight 700) #231F20, placed in the left negative space, aligned flush left. Subhead: \u201cHandcrafted Soy Milk for Your Daily Wellness\u201d in regular sans-serif #231F20, smaller size beneath headline. Accent line or dot element in #F06683 separates headline and subhead.",
#     "branding_visuals": "Placeholder for circular logo lockup in bottom right corner, logo in #231F20 on white circle with #F06683 stroke. Small brand tagline below logo in italic sans-serif #AEB0B2. A thin accent bar in #F06683 runs along the bottom edge of the frame.",
#     "texture_and_details": "Visible droplets of condensation on the bottle surface. Crisp texture of soybeans in the bowl, subtle wood grain on the countertop edge. Veins in the marble are softly defined.",
#     "target_audience": "Foodies/Gourmands",
#     "business_objective": "Introduce New Menu Item",
#     "content_niche_or_theme": "",
#     "brand_voice": "Luxurious"
# }
#     # "negative_elements": "No busy patterns or bright conflicting colors. Avoid harsh shadows or overly saturated greens. No unrelated props (e.g., cutlery, electronics) that distract from the soy milk theme.",
#     # "creative_reasoning": "By using a bright, minimalist kitchen setting and high-key natural light, we reinforce cleanliness and health\u2014key values for health-conscious millennials. The negative space and pink accent (#F06683) align with brand colors while providing room for informative, friendly text. Foreground soybeans and leaves underscore the plant-based soy niche. The photorealistic, magazine-style execution elevates product visibility and encourages engagement on Instagram with a cohesive, wellness-focused voice."

# visual_context.keys()


dict_keys(['main_subject', 'composition_and_framing', 'background_environment', 'foreground_elements', 'lighting_and_mood', 'color_palette', 'visual_style', 'promotional_text_visuals', 'branding_visuals', 'texture_and_details', 'target_audience', 'business_objective', 'content_niche_or_theme', 'brand_voice'])

In [ ]:
# @title Use Tool Search Result to Enhance Image Visual Context
client = OpenAI()

# Tool result from previous
# tool_search_result = response.output_text

gen_idx = 4
visual_context= generated_image_prompts[gen_idx]['visual_concept']
print(f"Generated Visual Context = {visual_context} \n")

expected_marketing_strat = suggested_marketing_strats[gen_idx]
print(f"Expected Marketing Strategies = {expected_marketing_strat} \n")

tool_search_result = f"""
   I couldn't find specific information about a company named "Eesoy."
   However, I can provide insights into the marketing strategies, brand positioning, and notable advertising campaigns of leading brands in the soy milk category.

  **Silk Soy Milk**

  - **Marketing Strategy**: Silk employs a comprehensive marketing approach that includes digital marketing, conventional advertising, and influencer partnerships. They utilize social media platforms to engage directly with consumers, answer queries, and gather feedback. Additionally, Silk integrates traditional marketing channels such as TV, magazine ads, and outdoor placements to reach a broader audience. ([case48.com](https://www.case48.com/case/Silk-Soy-Milk-A-Marketing-Strategy-11839?utm_source=openai))

  - **Brand Positioning**: Silk positions itself as a health-conscious alternative to dairy milk, emphasizing the nutritional benefits of soy. Their marketing often highlights the variety of flavors and health advantages, aiming to appeal to consumers seeking a wholesome beverage option. ([slideshare.net](https://www.slideshare.net/slideshow/silk-soymilk-advertising-plan/53906258?utm_source=openai))

  - **Notable Advertising Campaigns**: Silk has engaged in creative advertising initiatives, including collaborations with influencers and celebrities to promote their products. For instance, they have featured actors like Jeremy Renner in Super Bowl commercials, showcasing the product's benefits in an engaging manner. ([thebrandhopper.com](https://thebrandhopper.com/2024/10/12/a-deep-dive-into-the-marketing-strategies-of-danone-silk/?utm_source=openai))

  **Alpro**

  - **Marketing Strategy**: Alpro focuses on educational health positioning, aiming to motivate consumers to choose plant-based alternatives. Their strategy includes providing extensive recipes and cooking courses on their website, offering inspiration for incorporating plant-based products into daily meals. ([researchgate.net](https://www.researchgate.net/publication/375407704_Plant-Based_Milk_Alternatives_Consumer_Needs_and_Marketing_Strategies?utm_source=openai))

  - **Brand Positioning**: Alpro positions itself as a pioneer in plant-based dairy alternatives, emphasizing authenticity and a commitment to sustainability and health. They maintain transparency regarding ingredient sourcing and nutritional information to build trust with consumers. ([thebigmarketing.com](https://thebigmarketing.com/alpro-marketing-strategy-a-case-study/?utm_source=openai))

  - **Notable Advertising Campaigns**: Alpro's campaigns often focus on educating consumers about the benefits of plant-based nutrition and promoting a vegan lifestyle. They engage in strategic partnerships with organizations and influencers that align with their values, enhancing their market presence. ([thebigmarketing.com](https://thebigmarketing.com/alpro-marketing-strategy-a-case-study/?utm_source=openai))

  **Vitasoy**

  - **Marketing Strategy**: Vitasoy employs a 360-degree marketing approach, utilizing both digital and conventional marketing channels. They engage with consumers through social media, TV advertising, and point-of-sale displays, aiming to reach a wide audience. ([food-product-development-hannah.weebly.com](https://food-product-development-hannah.weebly.com/marketing-strategies-evaluation.html?utm_source=openai))

  - **Brand Positioning**: Vitasoy targets consumers with lactose, dairy, and gluten intolerances, positioning its products as healthy and suitable alternatives. ([food-product-development-hannah.weebly.com](https://food-product-development-hannah.weebly.com/marketing-strategies-evaluation.html?utm_source=openai))

  - **Notable Advertising Campaigns**: Vitasoy has released effective TV advertisements in China and Australia, highlighting the health benefits of their soy milk ranges. ([food-product-development-hannah.weebly.com](https://food-product-development-hannah.weebly.com/marketing-strategies-evaluation.html?utm_source=openai))

  **Oatly**

  - **Marketing Strategy**: Oatly's marketing approach is unconventional and bold, often challenging traditional dairy norms. They engage with their audience through witty and irreverent messaging, fostering a sense of trust and loyalty. Oatly also collaborates with influencers and celebrities who share their mission, extending their brand reach. ([latterly.org](https://www.latterly.org/oatly-marketing-strategy/?utm_source=openai))

  - **Brand Positioning**: Oatly positions itself as a young, cool, and rebellious brand, resonating with consumers who value sustainability and authenticity. They emphasize environmental responsibility and transparency in their messaging. ([latterly.org](https://www.latterly.org/oatly-marketing-strategy/?utm_source=openai))

  - **Notable Advertising Campaigns**: Oatly's campaigns often feature clever and irreverent messaging, such as their "It's Like Milk But Made For Humans" slogan. They also engage in unconventional marketing tactics, like using humor and sarcasm to challenge industry norms. ([latterly.org](https://www.latterly.org/oatly-marketing-strategy/?utm_source=openai))

  These brands exemplify diverse marketing strategies and brand positioning within the soy milk category, each catering to specific consumer needs and preferences.
"""

tool_search_result_enhanced =  f"User Expected Strategies {expected_marketing_strat} \n\n" + f"Online Tool Search Result : {tool_search_result} \n"

# Construct prompt to enhance the visual concept
enhance_prompt = f"""
  You are a visual branding assistant.

  Below is a base visual concept for a product image (JSON input), followed by a company profile and marketing strategy (or competitor strategy if the company is not found)..

  Your goal is to refine and elevate the base visual concept by:
  1. Aligns more closely with the brand's identity, values, and tone of voice.
  2. Enhancing visual communication to speak clearly to the company’s target audience and reflect its product positioning.
  3. Integrates subtle but powerful visual cues (e.g., lighting, texture, text placement) that reinforce the brand’s unique character—whether that’s luxury, natural simplicity, scientific rigor, or clinical precision.
  4. Translating abstract branding concepts into concrete design directions using design, photography, and art direction vocabulary (e.g., “matte finish,” “diffused natural light,” “editorial composition”).
  5. Evokes a compelling mood or emotional tone that supports the brand’s storytelling.
  6. Modifying or reworking the original layout or styling only if it reinforces brand differentiation or improves visual storytelling.
  7. Please return your final refined output in JSON format with these exact fields:
    [
      'company_name', 'product_type',
      'main_subject', 'composition_and_framing', 'background_environment', 'foreground_elements',
      'lighting_and_mood', 'color_palette', 'visual_style', 'promotional_text_visuals',
      'branding_visuals', 'texture_and_details',
      'target_audience', 'business_objective', 'content_niche_or_theme', 'brand_voice'
      'marketing_strategy', 'advertising_campaigns'
    ]

  --- Visual Concept ---
  {visual_context}

  ---  Company Profile or Competitor Marketing Strategy ---
  {tool_search_result_enhanced}

  If no company branding is found, use the competitor’s marketing strategy as a proxy and note this in the output.
  You may modify any part of the original visual concept if it results in a stronger alignment with the brand or improves competitive differentiation.

"""

# print(enhance_prompt)

visual_context_enhancement = client.responses.create(
    model='o4-mini',
    reasoning={
        "effort": "medium",
        "summary": "auto" # auto gives you the best available summary (detailed > auto > None)
    },
    input=[
        {"role": "system", "content": "You are a creative visual branding assistant."},
        {"role": "user", "content": enhance_prompt}
    ],
)

Generated Visual Context = {'main_subject': None, 'composition_and_framing': "Centered composition featuring the soy milk bottle as the primary subject, with surrounding contextual elements arranged in a triangular visual flow drawing attention back to the product. Dynamic diagonal balance created through placement of playful foreground elements (straw, fruit slices) aligned with the cap's orientation, while background textures subtly reinforce brand identity.", 'background_environment': 'Textured paper background with abstract leaf patterns in desaturated pastel tones, subtly echoing soy plant motifs. Integrated into the background are minimalist food illustrations (whole soybeans and almond shapes) that convey nutritional transparency. A faint gradient overlay in brand pink (#F06683) adds warmth while maintaining a clean white core space around the product.', 'foreground_elements': 'Colorful sliced strawberries and kiwis arranged in a semi-circle radiating from the bottle base, with 

In [ ]:
enhanced_prompt = visual_context_enhancement.output_text
print(f"Search Usage = Inputs : {visual_context_enhancement.usage.input_tokens} tokens, Outputs : {visual_context_enhancement.usage.output_tokens} tokens \n")
print(f"Reasoning Effort : {visual_context_enhancement.reasoning.effort} \n")
print(f"Output : {visual_context_enhancement.output_text}")

Search Usage = Inputs : 2351 tokens, Outputs : 2064 tokens 

Reasoning Effort : medium 

Output : {"company_name":"Eesoy","product_type":"Soy Milk","main_subject":"A premium glass bottle of Eesoy soy milk with a blush-pink cap and a clean, modern label front-and-center, angled slightly to reveal the nutritional panel.","composition_and_framing":"Asymmetrical rule-of-thirds: the bottle sits in the right third, pouring a stream of milk into a glass on the left. A subtle diagonal flow connects bottle, glass, and foreground elements, guiding the eye in a gentle triangular loop.","background_environment":"Warm beige, hand-torn craft paper with embossed soy leaf motifs and faint botanical line art in tonal ivory, evoking artisanal authenticity and natural origin.","foreground_elements":"Half-filled glass capturing the pouring motion, fresh strawberry and kiwi slices casually arranged, a bamboo straw angled toward the viewer, and a small chalkboard tile with handwritten “8 g Protein” in white

In [ ]:
# enhanced_prompt = {
#   "company_name": "Eesoy",
#   "product_type": "Premium Handcrafted Soy Milk",
#   "main_subject": "A frosted glass bottle of Eesoy soy milk with a minimalist matte-white label embossed in soft gold foil, capped with a natural cork top. Condensation is subtly visible to convey freshness without overwhelming the clean aesthetic.",
#   "color_palette": "Ivory (#F8F3ED), Warm Taupe (#D8CFC3), Soft Gold Foil (#C8A36D), Deep Charcoal Text (#2B2A29), Mint Green Accents (#A3C6A8).",
#   "visual_style": "High-end editorial photography with Scandinavian minimalism. Matte finish on surfaces, shallow depth of field to softly blur background texture, slight analogue film grain for warmth. Cinematic color grade emphasizing warm neutrals and gold foil highlights.",
#   "promotional_text_visuals": "Headline “PURE SOY CRAFT” in an elegant serif (weight medium) with soft gold foil effect, placed in the left negative space, flush left. Subhead “Experience Handcrafted Wellness” in a modern sans-serif (weight regular) in deep charcoal, smaller and right-aligned below the headline. A fine gold line separates the two texts.",
#   "branding_visuals": "Bottom right lockup of the circular Eesoy logo in matte gold on ivory, surrounded by a thin charcoal ring. Underneath, the tagline “Naturally Refined” in italic serif, taupe. A narrow gold foil accent bar runs along the bottom edge.",
#   "texture_and_details": "Subtle linen weave on the napkin, honed matte marble texture, fine film grain, delicate condensation droplets on the bottle. The porcelain dish’s unglazed foot reveals its hand-crafted origin.",
#   "target_audience": "Affluent health-conscious professionals and eco-luxury lifestyle seekers aged 25–45.",
#   "business_objective": "Launch and position Eesoy as a premium soy milk brand in the specialty dairy-alternative market.",
#   "content_niche_or_theme": "Premium wellness lifestyle and artisanal craft beverages.",
#   "brand_voice": "Sophisticated, authentic, wellness-focused.",
#   "marketing_strategy": "Borrowing Alpro’s educational health positioning as a proxy: emphasize transparency in sourcing (show farm-to-bottle stories), sustainability credentials, and nutritional benefits. Leverage digital channels for recipe content and expert interviews. Build trust through detailed ingredient storytelling and third-party certifications.",
#   "advertising_campaigns": "1. “Eesoy Crafted Recipes” video series with plant-based chefs. 2. Instagram “Eesoy Moments” UGC campaign inviting consumers to share their premium soy milk rituals. 3. Partnership with wellness influencers for “Morning Refined” sponsored content. 4. Print and digital ads in high-end lifestyle and culinary magazines highlighting farm-to-table authenticity."
# }
# print(enhanced_prompt)

#### Test OpenAI ImageGeneration / ImageEdit
- Using prompt from 'creative_prompt_to_image_gen'
- Tune parameters for more creativity

In [ ]:
image_client = OpenAI(api_key='')

# visual_context_enhancement = visual_context
# image_prompt=f"""
#   Create a new image with the following visual concept:
#   {enhanced_prompt}

#   This concept should serve as a creative foundation, but feel free to adapt, expand, or reinterpret elements as needed to enhance the final visual. You are encouraged to:
#   1. Add or remove details to heighten the emotional tone, storytelling, or visual clarity.
#   2. Introduce imaginative elements that strengthen the brand character—whether through setting, color palette, props, textures, or light.
#   3. Push the visual language further if it helps communicate the luxury, science-backed, clinical, or nature-driven identity more effectively.
#   4. Use artistic judgment to ensure the image feels refined, cohesive, and on-brand, even if it diverges from the original reference.
#   5. The result should be an elevated, creatively interpreted version of the visual direction, suitable for use in a high-quality brand campaign, mood board, or product showcase.
# """

image_prompt = f"""
  ---- IMAGE BRIEF: Visual Concept Reference with Company Branding -------
  {enhanced_prompt}

  --- OBJECTIVE ----
  Use the visual concept above as a launchpad, not a limitation.
  You are encouraged to reinterpret or reimagine the scene to better reflect the brand's emotional resonance, product identity, and lifestyle fit.
  Your goal is to create a visually striking, human-centered, and emotionally layered image that feels fresh, real, and brand-authentic.

  --- CREATIVE DIRECTION ----
  This is not a static product shot.
  It’s a moment pulled from a story—an image that speaks to the soul of the brand and how the product lives in the world.

  1. **Realistic and Relatable**
    - Ground the product in real-life contexts—how it’s used, where it lives, or how it fits naturally into someone’s daily lifestyle.
    - Prioritize authenticity and aspirational relatability.

  2. **Emotionally Resonant**
    - Capture a specific feeling or mood—freedom, calm, indulgence, confidence, curiosity, etc.—as experienced by real people.
    - Suggest human presence without showing a full face.
      (e.g., hands holding the product, a shoulder in frame, a figure from behind or partially out of focus)
    - Only Asian human presence (if included)
    - This maintains relatability while preserving versatility for marketing.
    - Use composition, props, environment, and lighting to evoke emotion and storytelling.

  3. **Lifestyle-Embedded**
    - Embed the product into a broader lifestyle narrative aligned with the target audience.
    - Include subtle lifestyle details and visual cues that reflect the company’s core values.

  4. ** Brand-Integrated (Marketing & Identity)**
    - Ensure the image includes the following branding elements:
      a. Company Logo
        - Integrate it naturally within the composition—on packaging, in the environment, or as a design overlay.
        - Use colors and placement consistent with the brand's visual identity.

      b. Slogan or Headline Text
        - Include key brand messaging or a promotional tagline as stylized overlay text if specified in the prompt.
        - Follow typographic and placement guidance provided in the enhanced prompt.
        - Overlay text should be legible, not intrusive—consider whitespace, padding, and image balance.

      c. Overlay Text Styling Guidance
        - Use a subtle text background overlay (e.g., semi-transparent white or dark box with 70–85% opacity) to ensure readability against complex visuals.
        - Apply soft drop shadow or blurred box glow under text for clarity if placed over bright areas.
        - Maintain padding around the text box (at least 5–8% of frame width) for elegant spacing.
        - Position overlay within the negative space or low-detail areas to avoid visual tension.
        - Keep text hierarchy clear (headline > subhead > taglines) with size and weight variation.

      d. Visual Messaging from Brand Strategy
        - Express brand values through visual styling, such as:
          Handcrafted quality: warm textures, artisanal props
          Natural simplicity: soft natural light, minimal composition
          Wellness orientation: calm ambiance, plant elements, refreshing tones
          Precision and clarity: clean backgrounds, sharp focus, orderly layout

      e. Taglines or Value Propositions (optional)
        - If space and composition allow, include subtle secondary text

 """


print(f"Image Prompt = {image_prompt}")

moderation = 'auto'
quality = 'high'

img = image_client.images.generate(
    model='gpt-image-1',
    prompt=image_prompt,
    n=1,
    quality=quality,
    size='1024x1024',
    moderation=moderation,
    output_format='png',
)

print("Prompt tokens:", img.usage.input_tokens_details.text_tokens)
print("Input images tokens:", img.usage.input_tokens_details.image_tokens)
print("Output image tokens:", img.usage.output_tokens)

Image Prompt = 
  ---- IMAGE BRIEF: Visual Concept Reference with Company Branding -------
  {"company_name":"Eesoy","product_type":"Soy Milk","main_subject":"A premium glass bottle of Eesoy soy milk with a blush-pink cap and a clean, modern label front-and-center, angled slightly to reveal the nutritional panel.","composition_and_framing":"Asymmetrical rule-of-thirds: the bottle sits in the right third, pouring a stream of milk into a glass on the left. A subtle diagonal flow connects bottle, glass, and foreground elements, guiding the eye in a gentle triangular loop.","background_environment":"Warm beige, hand-torn craft paper with embossed soy leaf motifs and faint botanical line art in tonal ivory, evoking artisanal authenticity and natural origin.","foreground_elements":"Half-filled glass capturing the pouring motion, fresh strawberry and kiwi slices casually arranged, a bamboo straw angled toward the viewer, and a small chalkboard tile with handwritten “8 g Protein” in white chal